# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [1]:
#!pip install -qU ragas==0.2.10

In [2]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
#import os
#import getpass

#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [5]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [6]:
#os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [7]:
!mkdir data

In [8]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31314    0 31314    0     0  46420      0 --:--:-- --:--:-- --:--:-- 46391


In [9]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70173    0 70173    0     0   418k      0 --:--:-- --:--:-- --:--:--  417k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [10]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [11]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/opt/anaconda3/envs/llmops-course/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [12]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [13]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [14]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

KnowledgeGraph(nodes: 14, relationships: 67)

We can save and load our knowledge graphs as follows.

In [15]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 14, relationships: 67)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [16]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [17]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

print(query_distribution)

[(SingleHopSpecificQuerySynthesizer(name='single_hop_specifc_query_synthesizer', llm=LangchainLLMWrapper(langchain_llm=ChatOpenAI(...)), generate_query_reference_prompt=QueryAnswerGenerationPrompt(instruction=Generate a single-hop query and answer based on the specified conditions (persona, term, style, length) and the provided context. Ensure the answer is entirely faithful to the context, using only the information directly from the provided context.### Instructions:
1. **Generate a Query**: Based on the context, persona, term, style, and length, create a question that aligns with the persona's perspective and incorporates the term.
2. **Generate an Answer**: Using only the content from the provided context, construct a detailed answer to the query. Do not add any information not included in or inferable from the context.
, examples=[(QueryCondition(persona=Persona(name='Software Engineer', role_description='Focuses on coding best practices and system design.'), term='microservices',


<style>
    .blue {
        background-color:rgb(93, 128, 158);
    }
</style>

<div class="blue">
    #### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

**Answer**
1. SingleHopSpecificQuerySynthesizer creates questions that can be answered directly from one single document or source. It's a "single hop" because the answer is contained within one place.
2. MultiHopAbstractQuerySynthesizer generates questions that require information from multiple documents or sources. It's "multi-hop" because it needs to jump between different pieces of information to form the answer. These questions are often more conceptual or analytical. They might ask for comparisons, summaries, or explanations.
3. MultiHopSpecificQuerySynthesizer also requires information from multiple sources. However, the answer is still relatively concrete and factual, even though it requires combining information. It's more about connecting the dots between facts.


</div>



Finally, we can use our `TestSetGenerator` to generate our testset!

In [18]:
print(kg.nodes)
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

[Node(id: 0022f4, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']), Node(id: 636c69, type: NodeType.DOCUMENT, properties: ['page_content', 'document_metadata', 'headlines', 'summary', 'summary_embedding']), Node(id: f2c525, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']), Node(id: bb4f36, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']), Node(id: 3059c7, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']), Node(id: 760c13, type: NodeType.CHUNK, properties: ['page_content', 'entities', 'themes']), Node(id: a4aaf7, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']), Node(id: 3c7766, type: NodeType.CHUNK, properties: ['page_content', 'entities', 'themes']), Node(id: ea96da, type: NodeType.CHUNK, properties: ['page_content', 'themes', 'entities']), Node(id: 246062, type: NodeType.CHUNK, properties: ['page_content', 'themes',

Generating Samples: 100%|██████████| 11/11 [00:39<00:00,  3.63s/it]


,user_input,reference_contexts,reference,synthesizer_name
0,"What OpenAI do with LLMs, they good or bad?",[Gullibility is the biggest unsolved problem C...,"A year ago, the only organization that had rel...",single_hop_specifc_query_synthesizer
1,What are some of the challenges associated wit...,[that they haven’t shared yet. Vibes Based Dev...,Some challenges associated with using OpenAI's...,single_hop_specifc_query_synthesizer
2,What were the key developments in Large Langua...,[Simon Willison’s Weblog Subscribe Stuff we fi...,2023 was the breakthrough year for Large Langu...,single_hop_specifc_query_synthesizer
3,What are some key insights about ChatGPT from ...,[issues in a way that’s surprisingly easy to f...,"In 2023, ChatGPT was a significant topic of di...",single_hop_specifc_query_synthesizer
4,Wht iz the Universal impact of longer input co...,[Prompt driven app generation is a commodity a...,Longer input context lengths dramatically incr...,single_hop_specifc_query_synthesizer
5,How has the concept of universal access to AI ...,[<1-hop>\n\nPrompt driven app generation is a ...,The concept of universal access to AI models e...,multi_hop_abstract_query_synthesizer
6,How have advancements in prompt-driven app gen...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in prompt-driven app generation a...,multi_hop_abstract_query_synthesizer
7,How has the pricing of AI models evolved in 20...,[<1-hop>\n\nPrompt driven app generation is a ...,"In 2024, the pricing of AI models has signific...",multi_hop_abstract_query_synthesizer
8,How do GPT-4 and GPT-4o differ in their capabi...,[<1-hop>\n\nAnother common technique is to use...,GPT-4 and GPT-4o differ significantly in their...,multi_hop_specific_query_synthesizer
9,How did the advancements in GPT-4 and the intr...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2024, the advancements in GPT-4, along with...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [19]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Generating Samples: 100%|██████████| 12/12 [01:14<00:00,  6.19s/it]


In [20]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What role has Microsoft Research played in the...,[Code may be the best application The ethics o...,Microsoft Research has contributed to the deve...,single_hop_specifc_query_synthesizer
1,Wht are the limtations of LLMs when it comes t...,[Based Development As a computer scientist and...,One of the great weaknesses of LLMs is their t...,single_hop_specifc_query_synthesizer
2,"What stuff we figured out about GPT-4 in 2023,...",[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, it was noted that while Large Languag...",single_hop_specifc_query_synthesizer
3,What ChatGPT do?,[easy to follow. The rest of the document incl...,ChatGPT is a topic frequently discussed in the...,single_hop_specifc_query_synthesizer
4,How do inference-scaling reasoning models cont...,[<1-hop>\n\nPrompt driven app generation is a ...,"Inference-scaling reasoning models, such as Op...",multi_hop_abstract_query_synthesizer
5,How have advancements in AI efficiency address...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI efficiency have significant...,multi_hop_abstract_query_synthesizer
6,How have advancements in AI models addressed t...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI models have addressed envir...,multi_hop_abstract_query_synthesizer
7,How have advancements in AI model efficiency a...,[<1-hop>\n\nPrompt driven app generation is a ...,Advancements in AI model efficiency have signi...,multi_hop_abstract_query_synthesizer
8,What are the advancements and limitations of G...,[<1-hop>\n\nSimon Willison’s Weblog Subscribe ...,"In 2023, GPT-4 and GPT-4o represented signific...",multi_hop_specific_query_synthesizer
9,How does the performance and ranking of Gemini...,[<1-hop>\n\nThose of us who understand this st...,Gemini 2.0 is ranked just ahead of DeepSeek v3...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [21]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [22]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [23]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [24]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [25]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [26]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [27]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [28]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [29]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

Finally, we can set-up our RAG LCEL chain!

In [30]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [31]:
rag_chain.invoke({"question" : "What are Agents?"})

'Agents are an infuriatingly vague term that generally refers to "AI systems that can go away and act on your behalf." There are different interpretations of the term, with some seeing AI agents as entities that perform tasks for users, akin to a travel agent, while others view them as large language models (LLMs) that have access to tools and can solve problems in a loop. However, there is a lack of clarity and consensus on what "agents" truly are, and they have not yet been realized in a practical sense.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4o as our evaluation LLM for our base Evaluators.

In [32]:
eval_llm = ChatOpenAI(model="gpt-4o")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [33]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

**Answer**

- `qa_evaluator`:  This evaluator is designed for general Question Answering (QA) tasks. It likely assesses how well the LLM's generated answer matches or aligns with the reference answer.

- `labeled_helpfulness_evaluator`: This evaluator is specifically judging the helpfulness of the LLM's response. It's using a "labeled_criteria" approach, meaning you're providing a specific definition of what "helpfulness" means in your context.

- `dope_or_nope_evaluator`: This evaluator is judging the style or tone of the LLM's response based on a very subjective criterion: "dopeness."

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'left-push-19' at:
https://smith.langchain.com/o/48c839b0-a50d-4327-8b62-d77e548f014c/datasets/0ec770e4-8c3f-48cc-93b7-46e957100ef7/compare?selectedSessions=e6db85a6-753d-4c41-8385-46fe36c4ae57




12it [02:24, 12.05s/it]


,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has the development of ChatGPT and similar...,The development of ChatGPT and similar models ...,None,The development of ChatGPT and similar models ...,1,0,0,4.582262,5e88c020-8df0-406d-843f-e1f011274851,c0cb50d9-0e92-4722-91c5-1d1be543113c
1,How have advancements in multi-modal LLMs and ...,Advancements in multi-modal LLMs have made it ...,None,Advancements in multi-modal LLMs and pricing e...,1,1,1,2.040925,b2ce2233-5c4f-443c-a138-a0823a5e8b7c,593dd527-4f53-410a-b4cd-9703dfab5afe
2,How does the performance and ranking of Gemini...,The context does not provide specific informat...,None,Gemini 2.0 is ranked just ahead of DeepSeek v3...,0,0,0,1.697677,3b45f4df-c771-4c7a-9d4d-d724bbe7c271,f4dd0601-1be9-4c2d-98c8-42698959a880
3,What are the advancements and limitations of G...,The advancements of GPT-4 and GPT-4o in 2023 i...,None,"In 2023, GPT-4 and GPT-4o represented signific...",1,0,0,4.081780,16dd7d89-bc95-4320-b237-0bb91beccaf8,b53cf452-cdee-49f1-9508-0fb7c01ffcc2
4,How have advancements in AI model efficiency a...,Advancements in AI model efficiency have led t...,None,Advancements in AI model efficiency have signi...,1,0,0,2.358697,d0ec20d1-1b9e-4a03-b94c-d9626759fcda,ba9f7e6e-46b1-4a72-8bd7-9dc3ece31b50
5,How have advancements in AI models addressed t...,Advancements in AI models have led to increase...,None,Advancements in AI models have addressed envir...,1,1,0,2.859630,da68d9e3-aad7-45c0-8b50-aa6d196edc44,c8d250b1-6927-4c56-b2dd-4af15f83aa49
6,How have advancements in AI efficiency address...,Advancements in AI efficiency have led to a si...,None,Advancements in AI efficiency have significant...,1,1,0,2.944301,aa86754d-899c-4d91-87d7-b377b6ba469d,1b142195-1963-4147-85c2-69d39b35bb41
7,How do inference-scaling reasoning models cont...,I don't know.,None,"Inference-scaling reasoning models, such as Op...",0,0,0,0.865954,3c38ec14-1e75-4b4a-8631-aee606c5c140,e13dbae8-138d-48ee-a654-43ff0cd0aa52
8,What ChatGPT do?,I don't know.,None,ChatGPT is a topic frequently discussed in the...,0,0,0,0.810274,ea9cae11-63d0-47c7-ae36-c3523958a063,1f559fb0-00b1-4039-8bee-b76d9cc78464
9,"What stuff we figured out about GPT-4 in 2023,...",I don't know.,None,"In 2023, it was noted that while Large Languag...",0,0,0,0.850602,e8395bcd-cf5b-4796-9f71-17f918b6ced3,edd5b060-ef62-4c51-9d63-ba97fd7db10d


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)


<style>
    .blue {
        background-color:rgb(93, 128, 158);
    }
</style>

<div class="blue">
    #### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

**ANSWER**

Larger chunks provide more context, which can be beneficial for understanding complex or multi-faceted queries. However, they also increase the chances of including irrelevant information, diluting the signal and making it harder for the model to focus on the most important parts.
Larger chunks can improve recall (the proportion of all relevant information that is retrieved) because they are more likely to contain all the necessary details. However, they can also decrease precision by including irrelevant information.   

**Impact on Computational Efficiency**:
1. Embedding Computation:
Larger chunks result in a smaller number of chunks, which means the embedding model needs to compute less embeddings. This can save the computational cost of the retrieval process.
2. Retrieval Speed:
A smallerer number of chunks can reduce the size of the retrieval index, which can improve the search process.
3. Generation Cost:
The LLM used for generation has a limited context window. If the retrieved chunks are too large, you might not be able to fit all the relevant context into the prompt, hindering the generation quality. On the other hand, if they are too small, it may require to retrieve more chunks.





</div>

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


<style>
    .blue {
        background-color:rgb(93, 128, 158);
    }
</style>

<div class="blue">
#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

**ANSWER**

Switching from the embedding model from text-embedding-3-small to text-embedding-3-large brings several effects, primarily related to embedding quality and computational cost.

1. Improved Semantic Understanding: text-embedding-3-large is designed to capture semantic meaning more accurately and comprehensively than the smaller model. 
2. Potentially Better Relevance: Because of the improved semantic understanding, the similarity scores calculated between query embeddings and document embeddings are likely to be more accurate.  This can lead to retrieving more relevant context, which is crucial for RAG performance.
3. Potentially Better Generalization: The larger model might generalize slightly better to unseen text, especially if your RAG application deals with a wide range of topics or language styles.

Increased Computational Cost: text-embedding-3-large will require significantly more computational resources than the smaller model.  This will likely lead to:
- Slower embedding computation: Generating embeddings for your documents and queries will take longer.
- Slower retrieval: Searching for similar embeddings in your vector database will also take longer.
- Increased storage requirements: The embeddings generated by the larger model will be larger, requiring more storage space.

</div>

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

"Agents, man, they're like those elusive AI systems that are supposed to act on your behalf. But here's the kicker: the term is super vague and people throw it around without a clear meaning. Some think of them as digital assistants or travel agents, while others see them as LLMs using tools to solve problems. But the truth is, despite all the hype, they’re still kinda stuck in the “coming soon” phase. And let’s be real, if they can’t figure out what’s true or false, how useful can they really be? That's the scoop on agents! 🕶️💻✨"

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'proper-rat-82' at:
https://smith.langchain.com/o/48c839b0-a50d-4327-8b62-d77e548f014c/datasets/0ec770e4-8c3f-48cc-93b7-46e957100ef7/compare?selectedSessions=98d88a50-ac09-481e-a253-400cd7d031c7




12it [02:51, 14.33s/it]


,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,How has the development of ChatGPT and similar...,"Yo, the development of ChatGPT and its multi-m...",None,The development of ChatGPT and similar models ...,1,0,1,3.497712,5e88c020-8df0-406d-843f-e1f011274851,e3c62d75-a53b-46d9-9836-7b6c2820d742
1,How have advancements in multi-modal LLMs and ...,"Yo, in 2024, multi-modal LLMs are flexing hard...",None,Advancements in multi-modal LLMs and pricing e...,1,1,1,2.567552,b2ce2233-5c4f-443c-a138-a0823a5e8b7c,ed5123c3-9783-482f-8873-d2d88c14a658
2,How does the performance and ranking of Gemini...,"Yo, check it! The context doesn't directly bre...",None,Gemini 2.0 is ranked just ahead of DeepSeek v3...,0,0,1,7.275951,3b45f4df-c771-4c7a-9d4d-d724bbe7c271,460b2bfe-90e2-4905-854d-9aaecad3e85e
3,What are the advancements and limitations of G...,"Alright, let’s break it down! \n\nIn 2023, GPT...",None,"In 2023, GPT-4 and GPT-4o represented signific...",1,0,1,6.553752,16dd7d89-bc95-4320-b237-0bb91beccaf8,ff853582-73f2-497b-a1b0-62dde91c4629
4,How have advancements in AI model efficiency a...,"Yo, check it out! Advancements in AI model eff...",None,Advancements in AI model efficiency have signi...,1,0,1,4.395162,d0ec20d1-1b9e-4a03-b94c-d9626759fcda,3709f918-3cc3-4480-b769-d1dc04c184d5
5,How have advancements in AI models addressed t...,"Yo, check it! Advancements in AI models have s...",None,Advancements in AI models have addressed envir...,1,1,1,4.021663,da68d9e3-aad7-45c0-8b50-aa6d196edc44,6bd19580-8e45-4468-b1de-b99730b45050
6,How have advancements in AI efficiency address...,"Yo, check it out! Advancements in AI efficienc...",None,Advancements in AI efficiency have significant...,1,1,1,2.632179,aa86754d-899c-4d91-87d7-b377b6ba469d,1c899211-a001-4a1b-93d0-fac909886e9d
7,How do inference-scaling reasoning models cont...,I don't know.,None,"Inference-scaling reasoning models, such as Op...",0,0,0,1.578719,3c38ec14-1e75-4b4a-8631-aee606c5c140,cc66955a-eaff-4624-a3c9-80637ebb0e21
8,What ChatGPT do?,"ChatGPT is like a super chill word calculator,...",None,ChatGPT is a topic frequently discussed in the...,0,0,1,1.934821,ea9cae11-63d0-47c7-ae36-c3523958a063,f6c62496-f8b6-4992-bd76-374e7af1f9ea
9,"What stuff we figured out about GPT-4 in 2023,...","Yo, in 2023, we figured out some pretty wild s...",None,"In 2023, it was noted that while Large Languag...",1,1,1,4.007049,e8395bcd-cf5b-4796-9f71-17f918b6ced3,6108f3ac-ac08-4b93-b858-4bd057f5a750


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

**ANSWER**

Comparing 2 experiment runs:

- Correctness: it didn't change much.
- Dope-ness: The answers were more dope on the 2nd chain, because a "dope" prompt augmentation was added.
- Helpfulness: The answers were more helpful on the 2nd chain. The reason is the embedding model is more powerful.
- Latency: P90 Latency is worse on the 2nd run. The reason may from the bigger embedding model which requires more computation.
- Total tokens and prompt tokens: the number of total tokens and prompt tokens were reduced on the 2nd run because the chunk size was increased.
- Completion tokens: The number of completion tokens was increased on the 2nd run probably because of the larger embeddiing model.


<img src="img/metrics_compare.jpg">